# Lispy

This notebook builds, step-by-step a simple lisp-ish interpreter. I made this in the process of applying for the [recurse center](https://www.recurse.com/apply/retreat) and kind of got a little carried away.

## Parser

First let's write a tokenizer/parser. We'll write code that takes some Lisp code and returns an abstract syntax tree. The AST should represent the structure of the code and the meaning of each token. For example give

```(first (list 1 (+ 2 3) 9))```

It should return a nested array like 

```["first", ["list", 1, ["+", 2, 3], 9]]```

In [1]:
delimiter = ['(', ')']
separator = ' ', '\n'
string_delim = ["'", '"']
result = []

def tokenize(code, nextToken='', result=None):
    if result is None:
        result = []        
    
    if len(code) == 0:
        return result or nextToken # deals with naked literals

    nextChar, rest = code[0], code[1:]
    
    if nextChar in delimiter:
        if nextChar == '(':
            return tokenize(rest, '', result + [nextChar])
        if nextChar == ')':
            if nextToken != '':
                return tokenize(rest, '', result + [nextToken, nextChar])
            else:
                return tokenize(rest, '', result + [nextChar])
    
    if nextChar in separator:
        if nextToken != '':
            return tokenize(rest, '', result + [nextToken])
        else:
            return tokenize(rest, '', result)
        
    # Properly tokenize strings
    if nextChar in string_delim:
        for i in range(len(rest)):
            if rest[i] == nextChar: # check for closing string delim
                return tokenize(rest[i+1:], '', result + ["\"{}\"".format(rest[:i])])
            
        raise Exception("Malformed input, unterminated string")
    
    return tokenize(rest, nextToken + nextChar, result)

In [2]:
def parser(tokens, ast=None, depth=0, balance=0):    
    if ast is None:
        ast = []
        
    if len(tokens) == 0:
        if depth == 0:
            assert balance == 0, "Unbalanced expression"
            
        return ast, tokens, balance
    
    nextToken, rest = tokens[0], tokens[1:]
    
    if nextToken in delimiter:
        if nextToken == '(':
            nextExpression, rest, new_balance = parser(rest, depth=depth+1, balance=balance+1)
            ast.append(nextExpression)
            
            return parser(rest, ast, depth=depth, balance=new_balance)
        if nextToken == ')':  
            balance -= 1
            assert balance >= 0, "Unbalanced expression"
            return ast, rest, balance
            
    return parser(rest, ast + [nextToken], depth=depth, balance=balance)

def parse(tokens):
    return parser(tokens)[0]

## Let's test the 💩 out of this tokenizer/parser

Otherwise this thing probably sucks

In [3]:
import unittest

literal_code = "1"
simple_code = "(first (list 1 (+ 2 3) 9))" 
challenge_code = """
(defun fibonacci (N)
  (if (or (zerop N) (= N 1))
      1
    (+ (fibonacci (- N 1)) (fibonacci (- N 2)))))
"""

malformed_code_l = '(()'
malformed_code_r = '())'

class TokenizerTests(unittest.TestCase):
    def test_literal(self):
        self.assertEqual(tokenize(literal_code), "1")
    
    def test_strings(self):
        self.assertEqual(tokenize("""(1 "hello world" 'wassup')"""), ['(', '1', '"hello world"', '"wassup"', ')'])

    def test_malformed_strings1(self):
        self.assertRaises(Exception, lambda: tokenize("""(1 "hello world)"""))

    def test_malformed_strings2(self):
        self.assertRaises(Exception, lambda: tokenize("""(1 'hello world)"""))

    def test_simple(self):
        self.assertEqual(tokenize(simple_code), ['(', 'first', '(', 'list', '1', '(', '+', '2', '3', ')', '9', ')', ')'])
        
    def test_challenge(self):
        self.assertEqual(tokenize(challenge_code), 
                         ['(', 'defun', 'fibonacci', '(', 'N', ')', '(', 'if', '(', 'or', '(', 'zerop', 'N', ')', 
                          '(', '=', 'N', '1', ')', ')', '1', '(', '+', '(', 'fibonacci', '(', '-', 'N', '1', ')', ')', 
                          '(', 'fibonacci', '(', '-', 'N', '2', ')', ')', ')', ')', ')'])

        
class ParserTests(unittest.TestCase):    
    def test_literal(self):
        self.assertEqual(parse(tokenize(literal_code)), ['1'])
    
    def test_simple(self):
        self.assertEqual(parse(tokenize(simple_code)), [['first', ['list', '1', ['+', '2', '3'], '9']]])
        
    def test_challenge(self):
        self.assertEqual(parse(tokenize(challenge_code)), 
                         [['defun', 'fibonacci', ['N'], 
                           ['if', ['or', ['zerop', 'N'], ['=', 'N', '1']], '1', 
                            ['+', ['fibonacci', ['-', 'N', '1']], ['fibonacci', ['-', 'N', '2']]]]]])
        
    def test_malformed_l(self):
        self.assertRaises(Exception, lambda: parse(tokenize(malformed_code_l)))
    
    def test_malformed_r(self):
        self.assertRaises(Exception, lambda: parse(tokenize(malformed_code_r)))
    
unittest.main(argv=[''], verbosity=1, exit=False);

...........
----------------------------------------------------------------------
Ran 11 tests in 0.003s

OK


# 🚀 Lisp Interpreter

Get a list of ASTs and run them 🏃‍♂️

In [4]:
1 + 1

2

In [5]:
a = "hello/world"

In [6]:
x, y = a.split('/')
x, y

('hello', 'world')

In [7]:
import re

module_delim = '/'

def resolve(ast, **env):
    # String literals
    if ast[0] in string_delim:
        return ast[1:-1], env

    # Ints
    if re.fullmatch('\d+', ast):
        return int(ast), env

    # Floats
    if re.fullmatch('\d+\.\d*', ast):
        return float(ast), env

    # Lastly check for symbols defined in the env
    
    # Check for modules, which keep their environments as nested dicts in the env
    if module_delim in ast and len(ast) > 1:
        module_name, symbol = ast.split(module_delim)
        
        if module_name not in env:
            raise Exception(f"Module not found - {module_name}")
        
        return env[module_name][symbol], env
    
    # Top level symbols
    return env[ast], env

def evaluate(ast, **env):
    if isinstance(ast, str):        
        return resolve(ast, **env)
    
    if isinstance(ast, list):
        assert len(ast) >= 1, "Empty expression"
        
        op_name = ast[0]        
        operation, _ = evaluate(op_name, **env) 

        if op_name in macros:
            # Macros are in charge of evalauting the ast of its body, and returning a tuple (result, env) potentially modifying the env
            return operation(*ast[1:], **env)
        
        # Otherwise we evaluate everything before giving it to the operation
        args = []
        for x in ast[1:]:
            arg, env = evaluate(x, **env) # Things could mutate the env during the evaluation, although seems like a terrible idea
            args.append(arg)
        
        # Normal procedure calls only get passed their args
        return operation(*args), env
    
    # Only used if the thing is already evaluated, this is probably wrong
    return ast, env

# 🏗 Functions & Macros

Now that we have an interpreter that works, we have to define the basic building blocks of the language. 

## Functions 
These are your run of the mill functions that you know and love. They look something like this:

```python
def my_func(*args):
    '''args have already been evaluated'''
    ...do something cool here...
```    

## Macros
These are a bit more interesting and more powerful. They get passed the _unevaluated_ `ast` as well as the environment, and they have to return a tuple of (result, env) where result has been evaluated.

```python
def my_macro(*ast, **env):
    '''args have already been evaluated'''
    ...do something very cool here...
```    

Things that full under macros include most of the actual syntax, including `if`, `or`, `when`, `defn`, `let`, etc.

## NB

Below I defined several foundational functions that all call out to Python. However, it's clear that many could just be `defn`d in the language itself as a "core" library. These is actually pretty cool, and is left as an exercise to the reader 😉.


In [13]:
from pathlib import Path

def addition_op(*args):
    acc = 0
    for x in args:
        acc += x
    return acc

def subtraction_op(*args):
    a, b = args[0], args[1]
    return a - b

def multiply_op(*args):
    acc = 1
    for x in args:
        acc *= x
    return acc

def range_op(*args):
    return list(range(*args))

def reduce_op(*args):
    f, init, xs = args[0], args[1], args[2]
    
    acc = init
    for x in xs:
        acc = f(acc, x)
    return acc

def list_op(*args):
    return list(args)

def cat_op(*args):
    acc = []
    for x in args:
        acc += x
    return acc

def len_op(*args):
    return len(*args)

def eq_op(*args):
    a, b = args[0], args[1]
    return a == b
    
def gt_op(*args):
    a, b = args[0], args[1]
    return a > b

def lt_op(*args):
    a, b = args[0], args[1]
    return a < b

def print_op(*args):
    print(*args)
    return None

def format_op(*args):
    format_str, xs = args[0], args[1:]
    print(format_str.format(*xs))    
    return None

def str_op(*args):
    acc = ''
    for x in args:
        acc += str(x)
    
    return acc

def type_op(*args):
    return type(args[0])


############
## Macros ##
############
def let_op(*ast, **env):
    '''Evaluates expression with modified environment
        
        (let (a 2 b 3)
            (+ a b))
    '''
    
    definitions, body = ast[0], ast[1]    

    assert len(definitions) % 2 == 0, "Let definitions must be in pairs"

    newEnv = dict(env) # copy the env to not overwrite it
    for i in range(0, len(definitions), 2):
        key, value = definitions[i], evaluate(definitions[i+1], **newEnv)[0]
        newEnv[key] = value
         
    return evaluate(body, **newEnv)[0], env

def def_op(*ast, **env):
    '''Allows for definition of bindings that can be referenced later. Mutates global env
        >> (def a 1)
        >> a
        1
    '''
    key, value_expr = ast[0], ast[1]
    value, _ = evaluate(value_expr, **env)
    
    env[key] = value
    return value, env

def fn_op(*ast, **env):
    '''Creates new anonymous function object and returns it'''
    signature, body = ast[0], ast[1]    
    
    def new_op(*args): 
        innerEnv = dict(env) # avoid mutating the global env
        for arg_name, arg_value in zip(signature, args):
            innerEnv[arg_name] = arg_value
        return evaluate(body, **innerEnv)[0] # We're defining a normal function (not a macro), thus we must not return the env coming from `evaluate`
    
    return new_op, env

def defn_op(*ast, **env):
    '''Defines a function and adds it to the global environment
    
        >> (defn my_sum (a b c) (+ a b c))
        >> (myfunc 1 2 3)
        6
    '''
    name, signature, body = ast[0], ast[1], ast[2]
    
    def new_op(*args): 
        innerEnv = {} # avoid mutating the global env
        for arg_name, arg_value in zip(signature, args):
            innerEnv[arg_name] = arg_value
        
        return evaluate(body, **dict(innerEnv, **env))[0]    
    
    env[name] = new_op
    return new_op, env

def if_op(*ast, **env):
    ''' Allows for conditional evaluation. The false branch does not get evaluated at all
    
    (if condition
        true_branch
        false_branch)
    '''
    condition, true_branch, false_branch = ast[0], ast[1], ast[2]
    
    if evaluate(condition, **env)[0]:
        return evaluate(true_branch, **env)
    else:
        return evaluate(false_branch, **env)

def do_op(*ast, **env):
    '''Evaluates all the expressions and returns the result of the last one'''
    result = None
    for expr in ast:
        result, env = evaluate(expr, **env)    
    return result, env
    
def when_op(*ast, **env):
    condition, branch = ast[0], ast[1]
    
    if evaluate(condition, **env)[0]:
        return evaluate(branch, **env)
        
    return None, env

def or_op(*ast, **env):
    '''Evaluates expressions left to r and returns the first truthy one'''
    
    for expression in ast:
        result, _ = evaluate(expression, **env)
        if result:
            return result, env
    
    return None, env

def import_op(*ast, **env):
    '''
    Allows evaluating a module by providing its location like so
    
    (import "./my/cool/mycode")
    
    This will evaluate all the code under the file "./my/cool/mycode.lspy".
    '''
    
    import_path = ast[0]
    module_name = Path(import_path).stem

    if len(ast) >= 2:
        if isinstance(ast[1], str):
            module_name = ast[1]
    
    with open(import_path[1:-1]) as import_file:
        import_source = import_file.read()
    
    _, module_env = interpret(import_source, env=dict(env), output_env=True)

    if module_name != '*':
        env[module_name] = {}

    # Add the new env entries added by the module_env to the top level env
    # The module definitions get put in a subdictionary, except if the module_name is '*'
    for symbol in module_env: 
        if symbol not in env:
            if module_name == '*':
                env[symbol] = module_env[symbol]
            else:
                env[module_name][symbol] = module_env[symbol]                
        
    return None, env
    
operators = {
    '+': addition_op,
    '-': subtraction_op,
    '*': multiply_op,
    'list': list_op,
    'range': range_op,
    'concat': cat_op,
    'reduce': reduce_op,
    'len': len_op,
    '=': eq_op,
    '>': gt_op,
    '<': lt_op,
    'True': True,
    'False': False,
    'None': None,
    'print': print_op,
    'format': format_op,
    'str': str_op,
    'type': type_op,
}

# This is a special dict that contains all macros
macros = {
    'let': let_op,
    'def': def_op,
    'fn': fn_op,
    'defn': defn_op,
    'if': if_op,
    'or': or_op,
    'do': do_op,
    'when': when_op,
    'or': or_op,   
    'import': import_op,
}

def lisp_eval(asts, env=None):
    if env is None:
        env = dict(operators, **macros) 

    result = None
    for ast in asts:
        result, env = evaluate(ast, **env) # potentially mutate env
    return result, env

def interpret(code, env=None, output_env=False):
    if env is None:
        env = dict(operators, **macros)
    
    if output_env is True:
        return lisp_eval(parse(tokenize(code)), env)
    return lisp_eval(parse(tokenize(code)), env)[0]

# 👋🏽 Hello World

Here are some code examples, and no language is complete without a good hello world to begin

In [14]:
interpret('''
(defn greeter (name)
    (print (str "hello " name)))

(greeter "world")

''')

hello world


## 🗺 Map

As mentioned before, we can actually implement new functions inside of our new cool language rather than calling out to python. One good example is `map`:

In [15]:
interpret('''
(defn map (f xs)
    (reduce (fn (acc x) (concat acc (list (f x)))) 
    (list) xs)
)

(map (fn (x) (* x x)) (range 1 10 2))
'''
)

[1, 9, 25, 49, 81]

## 🔃 Recursion!

We can even do recursion, we took especial care during the definition of `defn` that the environment where the new function is called contains a reference to itself 🙃. Given that this was originally made to apply for *Recurse*, I guess it was only fair I implemented it.

### 🐰 Factorial & Fibonacci 

Classic.

In [16]:
interpret('''
(defn map (f xs)
    (reduce (fn (acc x) (concat acc (list (f x)))) 
    (list) xs)
)

(defn factorial (n)
    (if (= n 0)
        1
        (* n (factorial (- n 1)))))
        
(print "Factorial" (map factorial (range 10)))        

(defn fib (n)
    (if (or (= n 0) (= n 1)) 
        1
        (+ (fib (- n 1)) (fib (- n 2)))))
        
(print "Fibonacci" (map fib (range 10)))
''')

Factorial [1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]
Fibonacci [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


# Imports!

We can now do imports that work like this:


In [17]:
interpret('''
(import './core.lspy' *)
(print (factorial 10))
''')

3628800


# Why U no REPL? 
(╯°□°)╯︵ ┻━┻

Wouldn't it be cool if we could run this interactively?!

It turns out it's way easy!

In [19]:
import traceback

def repl():
    env = dict(macros, **operators)

    while True:
        try:
            code = input('>>')
            asts = parse(tokenize(code))

            result = None
            for ast in asts:
                try:
                    result, env = evaluate(ast, **env)
                except Exception as e:
                    traceback.print_exc() # print exceptions nicely
                    continue
        except KeyboardInterrupt:
            print("Exiting...")
            return
        
        print(result)

In [20]:
repl()

>> (print "hello world")


hello world
None


>> (def a 1)


1


>> (print a)


...

>> (def a 2)


2


>> (print a)


2
None
Exiting...


>> 


# Conclusion

I'm actually kind of surprised of how simple and powerful lisp is. Laying down the original code took only a couple of hours and this first working version has been a couple afternoons worth of work. I'm a Clojure fan (many of the functions I implemented are simply emulating semantics from Clojure), but I had always assumed that implemeting a programming language was only for real hardcore programmers™️, and that to get event the most basic prototype it would take thousands of lines of code and many months of effort. After breaking it down into smal chunks that I could tackle one by one, I feel I got to build something that feels like a proper (toy) language in only a few hundred lines of Python. Now, I'm sure it has bugs as I haven't any kind of exhastive testing (evidently). 

One parting thought that is now stirring in the back of my head is what it takes to bootstrap a programming from scratch. I simply built a toy language on top of Python, and it's Python that's doing all the heavy lifting. Making a programming language from nothing seems like a totally different animal. In the case of Lisp, it was originally designed and implemented in ***1960*** (!!). It boggles my mind on how to even begin doing such thing, especially with the tools that were available back then. This has given me a new appreciation for programming languages, all the subtleties that come from making one from scratch, and an incredible respect for those who have paved the for us and on whom's shoulders we now stand on.